In [1]:
import os
import cv2
import time
import random
import logging
import easydict
import numpy as np
import pandas as pd
from tqdm import tqdm
from os.path import join as opj
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from PIL import Image
from natsort import natsorted

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms
from torch import Tensor
from torchvision.transforms import functional as F
import torch.cuda.amp as amp
from adamp import AdamP, SGDP

import warnings
warnings.filterwarnings('ignore')

In [2]:
args = easydict.EasyDict(
    {'exp_num':'0',
     
     # Path settings
     'data_path':'./open',
     'Kfold':5,
     'model_path':'label_results/',
     'image_type':'train_1024', 
     'class_num' : 88,

     # Model parameter settings
     'model_name':'regnety_040',
     'drop_path_rate':0.2,
     
     # Training parameter settings
     ## Base Parameter
     'img_size':512,
     'batch_size':16,
     'epochs':100,
     'optimizer':'Lamb',
     'initial_lr':5e-4,
     'weight_decay':1e-3,

     ## Augmentation
     'aug_ver':2,

     ## Scheduler (OnecycleLR)
     'scheduler':'Reduce',
     'warm_epoch':5,
     'max_lr':1e-3,

     ### Cosine Annealing
     'min_lr':5e-5,
     'tmax':145,

     ## etc.
     'patience': 7,
     'clipping':None,

     # Hardware settings
     'amp':True,
     'multi_gpu':True,
     'logging':False,
     'num_workers':4,
     'seed':42
     
     
    })

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"  # Set the GPUs 2 and 3 to use

# 데이터 증강

In [4]:
DATA_DIR = './open'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_df.csv'))
train_df = train_df.drop('index', axis=1)

In [5]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

# Random_Crop

In [6]:
def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]

    return crop

In [7]:
oslabel = list(train_df['label'].unique())


for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = cv2.resize(image, dsize=(1024, 1024))
            aug_img = get_random_crop(aug_img, 900, 900)
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_crop.png')
            save_name = f'{label}_{idx}_crop.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()

  0%|          | 0/88 [00:00<?, ?it/s]

cable-bent_wire
['10008.png', '11086.png', '11552.png', '11599.png', '13211.png', '13215.png', '13795.png']


  7%|▋         | 6/88 [00:00<00:08,  9.46it/s]

carpet-hole
['10010.png', '10307.png', '10641.png', '10711.png', '11673.png', '12260.png', '12432.png', '12823.png', '14030.png']


  8%|▊         | 7/88 [00:01<00:19,  4.15it/s]

...

 98%|█████████▊| 86/88 [00:46<00:01,  1.95it/s]

tile-rough
['11947.png', '12012.png', '12096.png', '12383.png', '12399.png', '12630.png', '12819.png', '13246.png']


 99%|█████████▉| 87/88 [00:46<00:00,  1.86it/s]

wood-liquid
['12106.png', '12178.png', '12610.png', '13183.png', '13259.png']


100%|██████████| 88/88 [00:47<00:00,  1.86it/s]


# Rotation

In [8]:
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h)) 
    return img

In [9]:
for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = rotation(image, 30)  
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_rotation.png')
            save_name = f'{label}_{idx}_rotation.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist() 

  0%|          | 0/88 [00:00<?, ?it/s]

cable-bent_wire
['10008.png', '11086.png', '11552.png', '11599.png', '13211.png', '13215.png', '13795.png', 'cable-bent_wire_0_crop.png', 'cable-bent_wire_1_crop.png', 'cable-bent_wire_2_crop.png', 'cable-bent_wire_3_crop.png', 'cable-bent_wire_4_crop.png', 'cable-bent_wire_5_crop.png', 'cable-bent_wire_6_crop.png']


  7%|▋         | 6/88 [00:01<00:16,  4.98it/s]

carpet-hole
['10010.png', '10307.png', '10641.png', '10711.png', '11673.png', '12260.png', '12432.png', '12823.png', '14030.png', 'carpet-hole_0_crop.png', 'carpet-hole_1_crop.png', 'carpet-hole_2_crop.png', 'carpet-hole_3_crop.png', 'carpet-hole_4_crop.png', 'carpet-hole_5_crop.png', 'carpet-hole_6_crop.png', 'carpet-hole_7_crop.png', 'carpet-hole_8_crop.png']


  8%|▊         | 7/88 [00:02<00:37,  2.13it/s]

...

 98%|█████████▊| 86/88 [01:38<00:02,  1.03s/it]

tile-rough
['11947.png', '12012.png', '12096.png', '12383.png', '12399.png', '12630.png', '12819.png', '13246.png', 'tile-rough_0_crop.png', 'tile-rough_1_crop.png', 'tile-rough_2_crop.png', 'tile-rough_3_crop.png', 'tile-rough_4_crop.png', 'tile-rough_5_crop.png', 'tile-rough_6_crop.png', 'tile-rough_7_crop.png']


 99%|█████████▉| 87/88 [01:39<00:01,  1.11s/it]

wood-liquid
['12106.png', '12178.png', '12610.png', '13183.png', '13259.png', 'wood-liquid_0_crop.png', 'wood-liquid_1_crop.png', 'wood-liquid_2_crop.png', 'wood-liquid_3_crop.png', 'wood-liquid_4_crop.png']


100%|██████████| 88/88 [01:40<00:00,  1.14s/it]


# Flip

In [10]:
for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = cv2.flip(image, 1)
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_flip1.png')
            save_name = f'{label}_{idx}_flip1.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()   

  0%|          | 0/88 [00:00<?, ?it/s]

cable-bent_wire
['10008.png', '11086.png', '11552.png', '11599.png', '13211.png', '13215.png', '13795.png', 'cable-bent_wire_0_crop.png', 'cable-bent_wire_0_rotation.png', 'cable-bent_wire_1_crop.png', 'cable-bent_wire_1_rotation.png', 'cable-bent_wire_2_crop.png', 'cable-bent_wire_2_rotation.png', 'cable-bent_wire_3_crop.png', 'cable-bent_wire_3_rotation.png', 'cable-bent_wire_4_crop.png', 'cable-bent_wire_4_rotation.png', 'cable-bent_wire_5_crop.png', 'cable-bent_wire_5_rotation.png', 'cable-bent_wire_6_crop.png', 'cable-bent_wire_6_rotation.png', 'cable-bent_wire_7_rotation.png', 'cable-bent_wire_8_rotation.png', 'cable-bent_wire_9_rotation.png', 'cable-bent_wire_10_rotation.png', 'cable-bent_wire_11_rotation.png', 'cable-bent_wire_12_rotation.png', 'cable-bent_wire_13_rotation.png']


  7%|▋         | 6/88 [00:02<00:32,  2.50it/s]

carpet-hole
['10010.png', '10307.png', '10641.png', '10711.png', '11673.png', '12260.png', '12432.png', '12823.png', '14030.png', 'carpet-hole_0_crop.png', 'carpet-hole_0_rotation.png', 'carpet-hole_1_crop.png', 'carpet-hole_1_rotation.png', 'carpet-hole_2_crop.png', 'carpet-hole_2_rotation.png', 'carpet-hole_3_crop.png', 'carpet-hole_3_rotation.png', 'carpet-hole_4_crop.png', 'carpet-hole_4_rotation.png', 'carpet-hole_5_crop.png', 'carpet-hole_5_rotation.png', 'carpet-hole_6_crop.png', 'carpet-hole_6_rotation.png', 'carpet-hole_7_crop.png', 'carpet-hole_7_rotation.png', 'carpet-hole_8_crop.png', 'carpet-hole_8_rotation.png', 'carpet-hole_9_rotation.png', 'carpet-hole_10_rotation.png', 'carpet-hole_11_rotation.png', 'carpet-hole_12_rotation.png', 'carpet-hole_13_rotation.png', 'carpet-hole_14_rotation.png', 'carpet-hole_15_rotation.png', 'carpet-hole_16_rotation.png', 'carpet-hole_17_rotation.png']


  8%|▊         | 7/88 [00:05<01:14,  1.08it/s]

...

 98%|█████████▊| 86/88 [03:18<00:04,  2.03s/it]

tile-rough
['11947.png', '12012.png', '12096.png', '12383.png', '12399.png', '12630.png', '12819.png', '13246.png', 'tile-rough_0_crop.png', 'tile-rough_0_rotation.png', 'tile-rough_1_crop.png', 'tile-rough_1_rotation.png', 'tile-rough_2_crop.png', 'tile-rough_2_rotation.png', 'tile-rough_3_crop.png', 'tile-rough_3_rotation.png', 'tile-rough_4_crop.png', 'tile-rough_4_rotation.png', 'tile-rough_5_crop.png', 'tile-rough_5_rotation.png', 'tile-rough_6_crop.png', 'tile-rough_6_rotation.png', 'tile-rough_7_crop.png', 'tile-rough_7_rotation.png', 'tile-rough_8_rotation.png', 'tile-rough_9_rotation.png', 'tile-rough_10_rotation.png', 'tile-rough_11_rotation.png', 'tile-rough_12_rotation.png', 'tile-rough_13_rotation.png', 'tile-rough_14_rotation.png', 'tile-rough_15_rotation.png']


 99%|█████████▉| 87/88 [03:20<00:02,  2.22s/it]

wood-liquid
['12106.png', '12178.png', '12610.png', '13183.png', '13259.png', 'wood-liquid_0_crop.png', 'wood-liquid_0_rotation.png', 'wood-liquid_1_crop.png', 'wood-liquid_1_rotation.png', 'wood-liquid_2_crop.png', 'wood-liquid_2_rotation.png', 'wood-liquid_3_crop.png', 'wood-liquid_3_rotation.png', 'wood-liquid_4_crop.png', 'wood-liquid_4_rotation.png', 'wood-liquid_5_rotation.png', 'wood-liquid_6_rotation.png', 'wood-liquid_7_rotation.png', 'wood-liquid_8_rotation.png', 'wood-liquid_9_rotation.png']


100%|██████████| 88/88 [03:22<00:00,  2.30s/it]


In [11]:
for label in tqdm(oslabel):
    if 'good' not in label:
        print(label)
        idx = 0
        one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
        images_list = natsorted(one_sample['file_name'])
        print(images_list)
        for _, image_name in enumerate(images_list):
            image = np.array(Image.open(opj('./open/train_add_data/', image_name)).convert('RGB'))
            
            aug_img = cv2.flip(image, 2)
            aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            save_path = opj('./open/train_add_data', f'{label}_{idx}_flip2.png')
            save_name = f'{label}_{idx}_flip2.png'
            idx += 1
            cv2.imwrite(save_path, aug_img)
            train_df.loc[len(train_df)] = [save_name] + one_sample.iloc[0][1:].values.tolist()   

  0%|          | 0/88 [00:00<?, ?it/s]

cable-bent_wire
['10008.png', '11086.png', '11552.png', '11599.png', '13211.png', '13215.png', '13795.png', 'cable-bent_wire_0_crop.png', 'cable-bent_wire_0_flip1.png', 'cable-bent_wire_0_rotation.png', 'cable-bent_wire_1_crop.png', 'cable-bent_wire_1_flip1.png', 'cable-bent_wire_1_rotation.png', 'cable-bent_wire_2_crop.png', 'cable-bent_wire_2_flip1.png', 'cable-bent_wire_2_rotation.png', 'cable-bent_wire_3_crop.png', 'cable-bent_wire_3_flip1.png', 'cable-bent_wire_3_rotation.png', 'cable-bent_wire_4_crop.png', 'cable-bent_wire_4_flip1.png', 'cable-bent_wire_4_rotation.png', 'cable-bent_wire_5_crop.png', 'cable-bent_wire_5_flip1.png', 'cable-bent_wire_5_rotation.png', 'cable-bent_wire_6_crop.png', 'cable-bent_wire_6_flip1.png', 'cable-bent_wire_6_rotation.png', 'cable-bent_wire_7_flip1.png', 'cable-bent_wire_7_rotation.png', 'cable-bent_wire_8_flip1.png', 'cable-bent_wire_8_rotation.png', 'cable-bent_wire_9_flip1.png', 'cable-bent_wire_9_rotation.png', 'cable-bent_wire_10_flip1.png', 

  7%|▋         | 6/88 [00:04<01:04,  1.27it/s]

carpet-hole
['10010.png', '10307.png', '10641.png', '10711.png', '11673.png', '12260.png', '12432.png', '12823.png', '14030.png', 'carpet-hole_0_crop.png', 'carpet-hole_0_flip1.png', 'carpet-hole_0_rotation.png', 'carpet-hole_1_crop.png', 'carpet-hole_1_flip1.png', 'carpet-hole_1_rotation.png', 'carpet-hole_2_crop.png', 'carpet-hole_2_flip1.png', 'carpet-hole_2_rotation.png', 'carpet-hole_3_crop.png', 'carpet-hole_3_flip1.png', 'carpet-hole_3_rotation.png', 'carpet-hole_4_crop.png', 'carpet-hole_4_flip1.png', 'carpet-hole_4_rotation.png', 'carpet-hole_5_crop.png', 'carpet-hole_5_flip1.png', 'carpet-hole_5_rotation.png', 'carpet-hole_6_crop.png', 'carpet-hole_6_flip1.png', 'carpet-hole_6_rotation.png', 'carpet-hole_7_crop.png', 'carpet-hole_7_flip1.png', 'carpet-hole_7_rotation.png', 'carpet-hole_8_crop.png', 'carpet-hole_8_flip1.png', 'carpet-hole_8_rotation.png', 'carpet-hole_9_flip1.png', 'carpet-hole_9_rotation.png', 'carpet-hole_10_flip1.png', 'carpet-hole_10_rotation.png', 'carpet

  8%|▊         | 7/88 [00:10<02:27,  1.82s/it]

...

 98%|█████████▊| 86/88 [06:35<00:08,  4.05s/it]

tile-rough
['11947.png', '12012.png', '12096.png', '12383.png', '12399.png', '12630.png', '12819.png', '13246.png', 'tile-rough_0_crop.png', 'tile-rough_0_flip1.png', 'tile-rough_0_rotation.png', 'tile-rough_1_crop.png', 'tile-rough_1_flip1.png', 'tile-rough_1_rotation.png', 'tile-rough_2_crop.png', 'tile-rough_2_flip1.png', 'tile-rough_2_rotation.png', 'tile-rough_3_crop.png', 'tile-rough_3_flip1.png', 'tile-rough_3_rotation.png', 'tile-rough_4_crop.png', 'tile-rough_4_flip1.png', 'tile-rough_4_rotation.png', 'tile-rough_5_crop.png', 'tile-rough_5_flip1.png', 'tile-rough_5_rotation.png', 'tile-rough_6_crop.png', 'tile-rough_6_flip1.png', 'tile-rough_6_rotation.png', 'tile-rough_7_crop.png', 'tile-rough_7_flip1.png', 'tile-rough_7_rotation.png', 'tile-rough_8_flip1.png', 'tile-rough_8_rotation.png', 'tile-rough_9_flip1.png', 'tile-rough_9_rotation.png', 'tile-rough_10_flip1.png', 'tile-rough_10_rotation.png', 'tile-rough_11_flip1.png', 'tile-rough_11_rotation.png', 'tile-rough_12_flip1

 99%|█████████▉| 87/88 [06:40<00:04,  4.41s/it]

wood-liquid
['12106.png', '12178.png', '12610.png', '13183.png', '13259.png', 'wood-liquid_0_crop.png', 'wood-liquid_0_flip1.png', 'wood-liquid_0_rotation.png', 'wood-liquid_1_crop.png', 'wood-liquid_1_flip1.png', 'wood-liquid_1_rotation.png', 'wood-liquid_2_crop.png', 'wood-liquid_2_flip1.png', 'wood-liquid_2_rotation.png', 'wood-liquid_3_crop.png', 'wood-liquid_3_flip1.png', 'wood-liquid_3_rotation.png', 'wood-liquid_4_crop.png', 'wood-liquid_4_flip1.png', 'wood-liquid_4_rotation.png', 'wood-liquid_5_flip1.png', 'wood-liquid_5_rotation.png', 'wood-liquid_6_flip1.png', 'wood-liquid_6_rotation.png', 'wood-liquid_7_flip1.png', 'wood-liquid_7_rotation.png', 'wood-liquid_8_flip1.png', 'wood-liquid_8_rotation.png', 'wood-liquid_9_flip1.png', 'wood-liquid_9_rotation.png', 'wood-liquid_10_flip1.png', 'wood-liquid_11_flip1.png', 'wood-liquid_12_flip1.png', 'wood-liquid_13_flip1.png', 'wood-liquid_14_flip1.png', 'wood-liquid_15_flip1.png', 'wood-liquid_16_flip1.png', 'wood-liquid_17_flip1.png'

100%|██████████| 88/88 [06:44<00:00,  4.59s/it]


# label 인코딩 후 증강된 csv 저장

In [12]:
train_labels = train_df["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]
train_labels

[72,
 15,
 72,
 76,
 3,
 76,
 15,
 55,
 4,
 72,
 22,
 33,
 76,
 53,
 9,
 76,
 46,
 76,
 52,
 59,
 82,
 40,
 15,
 54,
 15,
 33,
 3,
 76,
 52,
 68,
 76,
 33,
 9,
 31,
 52,
 56,
 84,
 63,
 9,
 9,
 76,
 21,
 72,
 76,
 55,
 63,
 55,
 45,
 76,
 72,
 21,
 21,
 76,
 2,
 52,
 63,
 28,
 21,
 3,
 76,
 3,
 76,
 28,
 84,
 33,
 55,
 40,
 40,
 55,
 55,
 21,
 84,
 72,
 86,
 40,
 63,
 33,
 40,
 84,
 21,
 15,
 84,
 50,
 21,
 28,
 33,
 52,
 33,
 75,
 31,
 55,
 72,
 47,
 45,
 3,
 60,
 10,
 72,
 84,
 45,
 15,
 9,
 33,
 45,
 47,
 21,
 45,
 33,
 63,
 40,
 18,
 18,
 85,
 13,
 59,
 40,
 68,
 21,
 16,
 68,
 55,
 52,
 45,
 55,
 55,
 33,
 55,
 21,
 84,
 63,
 55,
 84,
 44,
 23,
 33,
 72,
 33,
 55,
 72,
 55,
 72,
 55,
 43,
 72,
 69,
 63,
 72,
 9,
 84,
 40,
 28,
 28,
 33,
 83,
 38,
 63,
 13,
 62,
 9,
 3,
 40,
 33,
 9,
 9,
 21,
 58,
 45,
 55,
 21,
 59,
 63,
 9,
 3,
 33,
 84,
 63,
 68,
 33,
 28,
 40,
 9,
 15,
 83,
 52,
 45,
 33,
 21,
 63,
 28,
 15,
 63,
 45,
 76,
 76,
 84,
 40,
 76,
 57,
 45,
 72,
 32,
 55,
 68,
 33,


In [13]:
train_df['encoder_label'] = train_labels

In [14]:
train_df.to_csv('./open/train_df_add_data.csv')

# 증강된 데이터 셋 불러오기

In [15]:
DATA_DIR = './open'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_df_add_data.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test_df.csv'))

print(train_df.head())
print(test_df.head())
print(train_df.shape)
print(test_df.shape)

   Unnamed: 0  file_name       class state            label  encoder_label
0           0  10000.png  transistor  good  transistor-good             72
1           1  10001.png     capsule  good     capsule-good             15
2           2  10002.png  transistor  good  transistor-good             72
3           3  10003.png        wood  good        wood-good             76
4           4  10004.png      bottle  good      bottle-good              3
   index  file_name
0      0  20000.png
1      1  20001.png
2      2  20002.png
3      3  20003.png
4      4  20004.png
(13997, 6)
(2154, 2)


In [16]:
train_df['label'].unique()

array(['transistor-good', 'capsule-good', 'wood-good', 'bottle-good',
       'screw-good', 'cable-bent_wire', 'carpet-hole', 'hazelnut-good',
       'pill-pill_type', 'cable-good', 'metal_nut-scratch', 'pill-good',
       'screw-thread_side', 'zipper-fabric_border', 'leather-good',
       'pill-scratch', 'toothbrush-good', 'hazelnut-crack',
       'screw-manipulated_front', 'zipper-good', 'tile-good',
       'carpet-good', 'metal_nut-good', 'bottle-contamination',
       'grid-good', 'zipper-split_teeth', 'pill-crack', 'wood-combined',
       'pill-color', 'screw-thread_top', 'cable-missing_cable',
       'capsule-squeeze', 'zipper-rough', 'capsule-crack', 'capsule-poke',
       'metal_nut-flip', 'carpet-metal_contamination', 'metal_nut-color',
       'transistor-bent_lead', 'zipper-fabric_interior', 'leather-fold',
       'tile-glue_strip', 'screw-scratch_neck', 'screw-scratch_head',
       'hazelnut-cut', 'bottle-broken_large', 'bottle-broken_small',
       'leather-cut', 'cable-cut_

In [17]:
class_num = len(train_df.encoder_label.unique())
class_num

88

In [18]:
# Warmup Learning rate scheduler
from torch.optim.lr_scheduler import _LRScheduler
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

# Logging
def get_root_logger(logger_name='basicsr',
                    log_level=logging.INFO,
                    log_file=None):

    logger = logging.getLogger(logger_name)
    # if the logger has been initialized, just return it
    if logger.hasHandlers():
        return logger

    format_str = '%(asctime)s %(levelname)s: %(message)s'
    logging.basicConfig(format=format_str, level=log_level)

    if log_file is not None:
        file_handler = logging.FileHandler(log_file, 'w')
        file_handler.setFormatter(logging.Formatter(format_str))
        file_handler.setLevel(log_level)
        logger.addHandler(file_handler)

    return logger

class AvgMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.losses = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.losses.append(val)

In [19]:
class RandomRotation(transforms.RandomRotation):
    def __init__(self, p: float, degrees: int):
        super(RandomRotation, self).__init__(degrees)
        self.p = p

    def forward(self, img):
        if torch.rand(1) < self.p:
            fill = self.fill
            if isinstance(img, Tensor):
                if isinstance(fill, (int, float)):
                    fill = [float(fill)] * F.get_image_num_channels(img)
                else:
                    fill = [float(f) for f in fill]
            angle = self.get_params(self.degrees)

            img = F.rotate(img, angle, self.resample, self.expand, self.center, fill)
        return img

In [20]:
class Train_Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.img_path = df['file_name'].values
        self.target = df['encoder_label'].values 
        self.transform = transform

        print(f'Dataset size:{len(self.img_path)}')

    def __getitem__(self, idx):
        
        image = Image.open(opj('./open/train_add_data/', self.img_path[idx])).convert('RGB')
        image = self.transform(image)
        target = self.target[idx]

        return image, target

    def __len__(self):
        return len(self.img_path)

class Test_dataset(Dataset):
    def __init__(self, df, transform=None):
        self.img_path = df['file_name'].values
        self.transform = transform

        print(f'Test Dataset size:{len(self.img_path)}')

    def __getitem__(self, idx):

        image = Image.open(opj('./open/test/', self.img_path[idx])).convert('RGB')
        image = self.transform(image)

        return image

    def __len__(self):
        return len(self.img_path)

def get_loader(df, phase: str, batch_size, shuffle,
               num_workers, transform):
    if phase == 'test':
        dataset = Test_dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    else:
        dataset = Train_Dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                 pin_memory=True,
                                 drop_last=False)
    return data_loader

def get_train_augmentation(img_size, ver):
    if ver==1: # for validset
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])

    if ver == 2:

        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.3),
                transforms.RandomVerticalFlip(p=0.3),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                transforms.RandomAffine((-20,20)),
                RandomRotation(0.5, degrees=5),
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(), 
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
    
    
    return transform

In [21]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.model_ft = timm.create_model(args.model_name, pretrained=True,
                                    drop_path_rate=args.drop_path_rate,
                                    )
        num_head = self.model_ft.head.fc.in_features
        self.model_ft.head.fc = nn.Linear(num_head, 88)

#         self.model_ft = coatnet_3()
#         num_ftrs = self.model_ft.fc.in_features
#         self.model_ft.fc = nn.Linear(num_ftrs, args.class_num)
        
    def forward(self, x):
        out = self.model_ft(x)
        return out

class Network_test(nn.Module):
    def __init__(self, encoder_name):
        super().__init__()
        self.model_ft = timm.create_model(args.model_name, pretrained=True,
                                    drop_path_rate=args.drop_path_rate,
                                    )
        num_head = self.model_ft.head.fc.in_features
        self.model_ft.head.fc = nn.Linear(num_head, 88)

#         self.model_ft = coatnet_3()
#         num_ftrs = self.model_ft.fc.in_features
#         self.model_ft.fc = nn.Linear(num_ftrs, args.class_num)

    def forward(self, x):
        out = self.model_ft(x)
        return out

In [22]:
# # weighted crossentropy loss를 위한 weight 계산 함수


class_num = train_df.groupby(["encoder_label"])["encoder_label"].count().tolist()
class_weight = torch.tensor(np.max(class_num) / class_num).to("cuda", dtype=torch.float)
print(f"class_weight: {class_weight}")

class_weight: tensor([2.4437, 2.2216, 2.2216, 1.8708, 3.4911, 4.0729, 4.0729, 3.4911, 4.8875,
        1.7455, 4.0729, 4.8875, 4.8875, 2.0365, 2.2216, 1.7854, 2.2216, 2.0365,
        2.4437, 2.4437, 2.7153, 1.3964, 2.7153, 2.7153, 2.4437, 4.0729, 4.0729,
        4.0729, 1.4811, 4.0729, 4.0729, 2.7153, 2.7153, 1.0000, 2.7153, 2.7153,
        2.4437, 2.4437, 2.7153, 2.4437, 1.5959, 2.7153, 1.8798, 2.2216, 2.0365,
        1.7773, 2.0365, 1.8798, 2.7153, 2.2216, 1.8798, 2.4437, 1.4644, 4.8875,
        2.0365, 1.2219, 2.0365, 2.0365, 1.8798, 2.0365, 2.0365, 2.7153, 2.7153,
        1.7000, 3.0547, 2.7153, 3.0547, 1.6292, 6.5167, 4.8875, 4.8875, 4.8875,
        1.8357, 4.8875, 6.1094, 4.0729, 1.5830, 4.8875, 4.8875, 2.2216, 2.4437,
        3.0547, 2.7153, 3.0547, 1.6292, 2.7153, 2.7153, 3.0547],
       device='cuda:0')


In [23]:
class FocalLoss(nn.Module):
    """
    https://dacon.io/competitions/official/235585/codeshare/1796
    """

    def __init__(self, gamma=2.0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        # print(self.gamma)
        self.eps = eps
        self.ce = nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [24]:
class Trainer():
    def __init__(self, args, save_path):
        '''
        args: arguments
        save_path: Model 가중치 저장 경로
        '''
        super(Trainer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Logging
        log_file = os.path.join(save_path, 'log.log')
        self.logger = get_root_logger(logger_name='IR', log_level=logging.INFO, log_file=log_file)
        self.logger.info(args)
        # self.logger.info(args.tag)

        # Train, Valid Set load
        ############################################################################
        if args.step == 0 :
            df_train = pd.read_csv(opj(args.data_path, 'train_df_add_data.csv'))
        else :
            df_train = pd.read_csv(opj(args.data_path, f'train_{args.step}step.csv'))

#         if args.image_type is not None:
#             df_train['img_path'] = df_train['img_path'].apply(lambda x:x.replace('train_imgs', args.image_type))
#             df_train['img_path'] = df_train['img_path'].apply(lambda x:x.replace('test_imgs', 'test_1024'))

        kf = StratifiedKFold(n_splits=args.Kfold, shuffle=True, random_state=args.seed)
        for fold, (train_idx, val_idx) in enumerate(kf.split(range(len(df_train)), y=df_train['encoder_label'])):
            df_train.loc[val_idx, 'fold'] = fold
        val_idx = list(df_train[df_train['fold'] == int(args.fold)].index)

        df_val = df_train[df_train['fold'] == args.fold].reset_index(drop=True)
        df_train = df_train[df_train['fold'] != args.fold].reset_index(drop=True)

        # Augmentation
        self.train_transform = get_train_augmentation(img_size=args.img_size, ver=args.aug_ver)
        self.test_transform = get_train_augmentation(img_size=args.img_size, ver=1)

        # TrainLoader
        self.train_loader = get_loader(df_train, phase='train', batch_size=args.batch_size, shuffle=True,
                                       num_workers=args.num_workers, transform=self.train_transform)
        self.val_loader = get_loader(df_val, phase='train', batch_size=args.batch_size, shuffle=False,
                                       num_workers=args.num_workers, transform=self.test_transform)

        # Network
        self.model = Network(args).to(self.device)

        # Loss
#         self.criterion = nn.CrossEntropyLoss(weight=class_weight)
        self.criterion = FocalLoss()
#         self.criterion = CutMixCrossEntropyLoss(True)
        
        # Optimizer & Scheduler
#         self.optimizer = Lookahead(torch.optim.Adam(self.model.parameters(), lr=args.initial_lr), k=5, alpha=0.5)
        self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr)
        
        iter_per_epoch = len(self.train_loader)
        self.warmup_scheduler = WarmUpLR(self.optimizer, iter_per_epoch * args.warm_epoch)

        if args.scheduler == 'step':
            self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=args.milestone, gamma=args.lr_factor, verbose=True)
        elif args.scheduler == 'cos':
            tmax = args.tmax # half-cycle 
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max = tmax, eta_min=args.min_lr, verbose=True)
        elif args.scheduler == 'cycle':
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)
        else:
            self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, patience=5, factor=0.5, mode="max", verbose=True)
            
        if args.multi_gpu:
            self.model = nn.DataParallel(self.model).to(self.device)

        # Train / Validate
        best_loss = np.inf
        best_acc = 0
        best_epoch = 0
        early_stopping = 0
        start = time.time()
        for epoch in range(1, args.epochs+1):
            self.epoch = epoch

            if args.scheduler == 'cos':
                if epoch > args.warm_epoch:
                    self.scheduler.step()

            # Training
            train_loss, train_acc, train_f1 = self.training(args)

            # Model weight in Multi_GPU or Single GPU
            state_dict= self.model.module.state_dict() if args.multi_gpu else self.model.state_dict()

            # Validation
            val_loss, val_acc, val_f1 = self.validate(args, phase='val')

            # Save models
            if val_loss < best_loss:
                early_stopping = 0
                best_epoch = epoch
                best_loss = val_loss
                best_acc = val_acc
                best_f1 = val_f1

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),
                    }, os.path.join(save_path, 'best_model.pth'))
                self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
            else:
                early_stopping += 1

            # Early Stopping
            if early_stopping == args.patience:
                break

        self.logger.info(f'\nBest Val Epoch:{best_epoch} | Val Loss:{best_loss:.4f} | Val Acc:{best_acc:.4f} | Val F1:{best_f1:.4f}')
        end = time.time()
        self.logger.info(f'Total Process time:{(end - start) / 60:.3f}Minute')

    # Training
    def training(self, args):
        self.model.train()
        train_loss = AvgMeter()
        train_acc = 0
        preds_list = []
        targets_list = []

        scaler = grad_scaler.GradScaler()
        
        for i, (images, targets) in enumerate(tqdm(self.train_loader)):
            images = torch.tensor(images, device=self.device, dtype=torch.float32)
            targets = torch.tensor(targets, device=self.device, dtype=torch.long)
            
            if self.epoch <= args.warm_epoch:
                self.warmup_scheduler.step()

            self.model.zero_grad(set_to_none=True)
    
            if args.amp:
                with autocast():
                    preds = self.model(images)
                    loss = self.criterion(preds, targets)
                    
                scaler.scale(loss).backward()

                # Gradient Clipping
                if args.clipping is not None:
                    scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)

                scaler.step(self.optimizer)
                scaler.update()

            else:
                preds = self.model(images)
                loss = self.criterion(preds, targets)
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)
                self.optimizer.step()

            if args.scheduler == 'cycle':
                if self.epoch > args.warm_epoch:
                    self.scheduler.step()

            # Metric
            train_acc += (preds.argmax(dim=1) == targets).sum().item()
            preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
            targets_list.extend(targets.cpu().detach().numpy())
            # log
            train_loss.update(loss.item(), n=images.size(0))

        train_acc /= len(self.train_loader.dataset)
        train_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

        self.logger.info(f'Epoch:[{self.epoch:03d}/{args.epochs:03d}]')
        self.logger.info(f'Train Loss:{train_loss.avg:.3f} | Acc:{train_acc:.4f} | F1:{train_f1:.4f}')
        return train_loss.avg, train_acc, train_f1
            
    # Validation or Dev
    def validate(self, args, phase='val'):
        self.model.eval()
        with torch.no_grad():
            val_loss = AvgMeter()
            val_acc = 0
            preds_list = []
            targets_list = []

            for i, (images, targets) in enumerate(self.val_loader):
                images = torch.tensor(images, device=self.device, dtype=torch.float32)
                targets = torch.tensor(targets, device=self.device, dtype=torch.long)

                preds = self.model(images)
                loss = self.criterion(preds, targets)

                # Metric
                val_acc += (preds.argmax(dim=1) == targets).sum().item()
                preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
                targets_list.extend(targets.cpu().detach().numpy())

                # log
                val_loss.update(loss.item(), n=images.size(0))
            val_acc /= len(self.val_loader.dataset)
            val_f1 = f1_score(np.array(targets_list), np.array(preds_list), average='macro')

            self.logger.info(f'{phase} Loss:{val_loss.avg:.3f} | Acc:{val_acc:.4f} | F1:{val_f1:.4f}')
        return val_loss.avg, val_acc, val_f1

In [25]:
def main(args):
    print('<---- Training Params ---->')
    
    # Random Seed
    seed = args.seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

    save_path = os.path.join(args.model_path, (args.exp_num).zfill(3))
    
    # Create model directory
    os.makedirs(save_path, exist_ok=True)
    Trainer(args, save_path)

    return save_path

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 2


In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sub = pd.read_csv('./open/sample_submission.csv')
df_train = pd.read_csv('./open/train_df_add_data.csv')
df_test = pd.read_csv('./open/test_df.csv')

In [28]:
def predict(encoder_name, test_loader, device, model_path):
    model = Network_test(encoder_name).to(device)
    model.load_state_dict(torch.load(opj(model_path, 'best_model.pth'))['state_dict'])
    model.eval()
    preds_list = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = torch.as_tensor(images, device=device, dtype=torch.float32)
            preds = model(images)
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(preds_list)

def ensemble_5fold(model_path_list, test_loader, device):
    predict_list = []
    for model_path in model_path_list:
        prediction = predict(encoder_name= 'regnety_040', test_loader = test_loader, device = device, model_path = model_path)
        predict_list.append(prediction)
    ensemble = (predict_list[0] + predict_list[1] + predict_list[2] + predict_list[3] + predict_list[4])/len(predict_list)

    return ensemble


In [ ]:
args.step = 0
models_path = []
for s_fold in range(5): # 5fold
    args.fold = s_fold
    args.exp_num = str(s_fold)
    save_path = main(args)
    models_path.append(save_path)

2022-05-03 14:10:35,657 INFO: {'exp_num': '0', 'data_path': './open', 'Kfold': 5, 'model_path': 'label_results/', 'image_type': 'train_1024', 'class_num': 88, 'model_name': 'regnety_040', 'drop_path_rate': 0.2, 'img_size': 512, 'batch_size': 16, 'epochs': 100, 'optimizer': 'Lamb', 'initial_lr': 0.0005, 'weight_decay': 0.001, 'aug_ver': 2, 'scheduler': 'Reduce', 'warm_epoch': 5, 'max_lr': 0.001, 'min_lr': 5e-05, 'tmax': 145, 'patience': 7, 'clipping': None, 'amp': True, 'multi_gpu': True, 'logging': False, 'num_workers': 4, 'seed': 42, 'step': 0, 'fold': 0}


<---- Training Params ---->
Dataset size:11197
Dataset size:2800


2022-05-03 14:10:36,000 INFO: Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_040-f0d569f9.pth)
100%|██████████| 700/700 [04:28<00:00,  2.60it/s]
2022-05-03 14:15:05,083 INFO: Epoch:[001/100]
2022-05-03 14:15:05,084 INFO: Train Loss:4.249 | Acc:0.0608 | F1:0.0325
2022-05-03 14:15:41,916 INFO: val Loss:3.394 | Acc:0.2600 | F1:0.0921
2022-05-03 14:15:42,632 INFO: -----------------SAVE:1epoch----------------
100%|██████████| 700/700 [06:37<00:00,  1.76it/s]
2022-05-03 14:22:19,778 INFO: Epoch:[002/100]
2022-05-03 14:22:19,779 INFO: Train Loss:3.083 | Acc:0.2633 | F1:0.1158
2022-05-03 14:22:55,472 INFO: val Loss:1.827 | Acc:0.3336 | F1:0.1426
2022-05-03 14:22:56,206 INFO: -----------------SAVE:2epoch----------------
100%|██████████| 700/700 [06:29<00:00,  1.80it/s]
2022-05-03 14:29:25,717 INFO: Epoch:[003/100]
2022-05-03 14:29:25,718 INFO: Train Loss:2.102 | Acc:0.3230 | F1:0.2026
2022-05-03 14:30:00,708 INFO: val

2022-05-03 17:34:59,712 INFO: Train Loss:0.037 | Acc:0.9764 | F1:0.9759
2022-05-03 17:35:35,766 INFO: val Loss:0.004 | Acc:0.9954 | F1:0.9952
100%|██████████| 700/700 [06:38<00:00,  1.76it/s]
2022-05-03 17:42:14,259 INFO: Epoch:[030/100]
2022-05-03 17:42:14,260 INFO: Train Loss:0.038 | Acc:0.9754 | F1:0.9748
2022-05-03 17:42:50,055 INFO: val Loss:0.014 | Acc:0.9925 | F1:0.9923
100%|██████████| 700/700 [06:38<00:00,  1.76it/s]
2022-05-03 17:49:28,836 INFO: Epoch:[031/100]
2022-05-03 17:49:28,837 INFO: Train Loss:0.036 | Acc:0.9770 | F1:0.9770
2022-05-03 17:50:04,309 INFO: val Loss:0.004 | Acc:0.9961 | F1:0.9953
100%|██████████| 700/700 [06:22<00:00,  1.83it/s]
2022-05-03 17:56:26,788 INFO: Epoch:[032/100]
2022-05-03 17:56:26,789 INFO: Train Loss:0.031 | Acc:0.9793 | F1:0.9790
2022-05-03 17:57:01,802 INFO: val Loss:0.010 | Acc:0.9925 | F1:0.9910
100%|██████████| 700/700 [06:17<00:00,  1.85it/s]
2022-05-03 18:03:19,188 INFO: Epoch:[033/100]
2022-05-03 18:03:19,189 INFO: Train Loss:0.032 |

<---- Training Params ---->
Dataset size:11197
Dataset size:2800


2022-05-03 19:29:13,969 INFO: Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-regnet/regnety_040-f0d569f9.pth)
100%|██████████| 700/700 [06:39<00:00,  1.75it/s]
2022-05-03 19:35:53,672 INFO: Epoch:[001/100]
2022-05-03 19:35:53,673 INFO: Train Loss:4.243 | Acc:0.0597 | F1:0.0310
2022-05-03 19:36:29,289 INFO: val Loss:3.382 | Acc:0.2686 | F1:0.0922
2022-05-03 19:36:30,159 INFO: -----------------SAVE:1epoch----------------
100%|██████████| 700/700 [06:24<00:00,  1.82it/s]
2022-05-03 19:42:54,520 INFO: Epoch:[002/100]
2022-05-03 19:42:54,521 INFO: Train Loss:3.081 | Acc:0.2649 | F1:0.1156
2022-05-03 19:43:28,108 INFO: val Loss:1.813 | Acc:0.3211 | F1:0.1311


In [ ]:
img_size = 512

test_transform = get_train_augmentation(img_size=img_size, ver=1)
test_dataset = Test_dataset(df_test, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [ ]:
# models_path = ['./label_results/000', './label_results/001', './label_results/002', './label_results/003', './label_results/004']

In [ ]:
ensemble = ensemble_5fold(models_path, test_loader, device)

In [ ]:
f_pred = ensemble.argmax(axis=1).tolist()
f_pred

In [ ]:
train_y = pd.read_csv("./open/train_df_add_data.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}


In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
f_result

In [ ]:
submission = pd.read_csv("./open/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv("./submission/label_result_add_0503_label_regnety_040.csv", index = False)